In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN1D(nn.Module):
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.fc1 = nn.Linear(128 * 3838, 100)  # Adjust the input size based on pooling and convolution
        self.fc2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 128 * 3838)  # Adjust based on the input size
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN1D(num_classes=10)  # Adjust num_classes as needed
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=100, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, num_classes)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 100).to(x.device)  # Initial hidden state
        c0 = torch.zeros(2, x.size(0), 100).to(x.device)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Get the output from the last time step
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

model = LSTMModel(num_classes=10)  # Adjust num_classes as needed
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN_LSTM(nn.Module):
    def __init__(self, num_classes):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.lstm = nn.LSTM(input_size=64, hidden_size=100, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.permute(0, 2, 1)  # Change the shape for LSTM (batch, seq, feature)
        h0 = torch.zeros(2, x.size(0), 100).to(x.device)  # Initial hidden state
        c0 = torch.zeros(2, x.size(0), 100).to(x.device)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Get the output from the last time step
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

model = CNN_LSTM(num_classes=10)  # Adjust num_classes as needed
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class TransformerModel(nn.Module):
    def __init__(self, num_classes, d_model=128, nhead=8, num_layers=4):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(1, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc1 = nn.Linear(d_model * 7680, 100)  # Adjust the input size based on d_model
        self.fc2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Transformer expects input shape (seq_len, batch, feature)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)
        x = x.contiguous().view(x.size(0), -1)  # Flatten for the fully connected layer
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = TransformerModel(num_classes=10)  # Adjust num_classes as needed
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
